In [52]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
import shutil

In [53]:
sample_size = -1
def load_images_from_folder(folder):
    images = {}
    for filename in os.listdir(folder)[:sample_size]:
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images[filename] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return images

In [54]:
def get_segments(name,image):

    if image is None:
        print("Could not read the image.")
        return 0
    # Preprocess the image (e.g., apply Gaussian blur)
    image = cv2.GaussianBlur(image, (5, 5), 0)

    # Apply adaptive thresholding
    image = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 10)

    # Perform adaptive dilation
    height, width = image.shape[:2]

    # Customize the kernel size based on the image dimensions and line characteristics
    kernel_height = 1  # Adjust as needed
    kernel_width = int(width)    # Adjust as needed
    kernel = np.full((kernel_height, kernel_width),255, np.uint8)
    img_dilation = cv2.dilate(image, kernel, iterations=10)

    # Find contours
    contours, _ = cv2.findContours(img_dilation, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        print("No contours found.")
        return 0

    if len(contours) == 1:
        # Get bounding box
        x, y, w, h = cv2.boundingRect(contours[0])
        # Extract ROI
        roi = image[y:y + h, x:x + w]
        # Save and display ROI
        images[name] = roi

        if not os.path.exists('words_dataset'):
            os.makedirs('words_dataset')
        
        for f in os.listdir('words_dataset'):
            if os.path.isfile('words_dataset/'+f): 
                os.remove('words_dataset/'+f)
            elif os.path.isdir('words_dataset/'+f):
                shutil.rmtree('words_dataset/'+f)

        cv2.imwrite('words_dataset/'+ name,255 - roi)

    return 1

In [55]:
# segmenting lines
images = load_images_from_folder('dataset')
filenames = os.listdir('dataset')
print(len(filenames))

for name in filenames[:sample_size]:
    get_segments(name,images.pop(name))
print(len(images))

330961
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
No contours found.
27747


In [56]:
# Open the CSV file
labels = {}
chars_labels = {}
with open('written_name_train_v2.csv', 'r') as file:
    # Create a CSV reader object
    csv_reader = csv.reader(file)

    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Access data in each row
        if row[0] in images.keys():
            labels[row[0]] = str(row[1]).replace(" ","")

In [57]:
def segment_characters(name, segment):
    if len(segment.shape) == 3:
        segment = cv2.cvtColor(segment, cv2.COLOR_BGR2GRAY)
    
    _, segment = cv2.threshold(segment, 128, 255, cv2.THRESH_BINARY)

    # Find contours in the binary image
    contours, _ = cv2.findContours(segment, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sort contours based on x-coordinate
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])
    if len(contours) != len(labels[name]):
        return 0

    # heights = [cv2.boundingRect(contour)[3] for contour in contours]
    # avgheight = sum(heights)/len(heights)
    
    # widths = [cv2.boundingRect(contour)[2] for contour in contours]
    # avgwidth = sum(widths)/len(widths)

    factor = 1
    prev_end = None
    
    # Iterate through each contour (presumed to be a character) and save as individual images
    for i, contour in enumerate(contours):

        x, y, w, h = cv2.boundingRect(contour)

        # Ignore contours that are too small

        # if w < avgwidth/10 and h < avgheight/10:
        #     continue

        # if prev_end is None:
        #     prev_end = x + w

        # if x > prev_end + factor * avgwidth:
        #     print("Space detected")

        character_image = segment[y:y + h, x:x + w]

        # color to Gray scale character_image
                
        cv2.imwrite('chars_dataset/'+str(i)+name, 255 - character_image)
        chars[str(i)+name] = np.where(character_image > 50 , 0,1)
        chars_labels[str(i)+name] = labels[name][i]

In [58]:
chars = {}
for name in images.keys():
    segment_characters(name,images[name])
print(len(chars))

68751


In [59]:
trainfile = 'char_train.csv'
testfile = 'char_test.csv'

# the height and width of our final image
fnl_hyt = 100
fnl_wdt = 100
pad_value = 1

if(os.path.exists(trainfile) and os.path.isfile(trainfile)): 
  os.remove(trainfile) 
if(os.path.exists(testfile) and os.path.isfile(testfile)): 
  os.remove(testfile) 

In [60]:
def crop(image):
    img_wdt = len(image[0])
    img_hyt = len(image)
    non_white_pixels = np.argwhere(image != pad_value)

    top_mrgn = 0
    bot_mrgn = img_hyt-1
    left_mrgn = 0
    right_mrgn = img_wdt-1

    # method 1****************************************************

    top_mrgn, left_mrgn = non_white_pixels.min(axis=0)
    bot_mrgn, right_mrgn = non_white_pixels.max(axis=0)

    # method 2*****************************************************
    # for i in range(img_hyt):
    #     if crop_cond(image[i]):
    #         top_mrgn = i
    #         break
    
    # for i in range(img_hyt):
    #     if crop_cond(image[img_hyt - i - 1]):
    #         bot_mrgn = img_hyt - i - 1
    #         break
    
    # t_image = image.T
    # for i in range(img_wdt):
    #     if crop_cond(t_image[i]):
    #         left_mrgn = i
    #         break
    
    # for i in range(img_wdt):
    #     if crop_cond(t_image[img_wdt - i - 1]):
    #         bot_mrgn = img_wdt - i - 1
    #         break
    
    return image[top_mrgn:bot_mrgn+1 , left_mrgn:right_mrgn+1]

In [61]:
# the value which we are padding
def pad(image):
    img_hyt = len(image)
    img_wdt = len(image[0])

    pad_hyt = img_hyt
    pad_wdt = img_wdt

    while(pad_hyt%10 != 0):
        pad_hyt += 1

    while(pad_wdt%10 != 0):
        pad_wdt += 1

    if pad_wdt > pad_hyt:
        pad_hyt = pad_wdt
    else :
        pad_wdt = pad_hyt
    
    top_pad = np.zeros(((pad_hyt-img_hyt)//2, img_wdt)) + pad_value
    bot_pad = np.zeros(((pad_hyt-img_hyt+1)//2 , img_wdt)) + pad_value
    left_pad = np.zeros((pad_hyt, (pad_wdt-img_wdt)//2)) + pad_value
    right_pad = np.zeros((pad_hyt, (pad_wdt-img_wdt+1)//2)) + pad_value

    # Stack the padding arrays and the image
    return np.hstack((left_pad, np.vstack((top_pad, image,bot_pad)) ,right_pad))

In [62]:
from scipy.ndimage import zoom  
def reshape(image):
    return np.where(zoom(image,(fnl_hyt/len(image), fnl_wdt/len(image[0])) , order = 1) > 0.5,1,0)

In [63]:
def trim(image, avg):
    int_fill = 1 - (np.sum(image) / (image.shape[0] * image.shape[1]))
    fill = int_fill

    if fill > avg :
        while fill > avg:
            change = np.argwhere((image - np.roll(image, 1, axis=0)) +
                                (image - np.roll(image, 1, axis=1)) +
                                (image - np.roll(image, -1, axis=0)) +
                                (image - np.roll(image, -1, axis=1)) < 0)

            image[change[:, 0], change[:, 1]] = pad_value
            tfill = 1 - (np.sum(image) / (image.shape[0] * image.shape[1]))

            if tfill >= fill:
                raise ValueError("Error: Unable to trim the image further.")

            fill = tfill
    
    else :
        image = 1 - image
        fill = 1 - fill
        while fill >= 1 - avg:
            change = np.argwhere((image - np.roll(image, 1, axis=0)) +
                                (image - np.roll(image, 1, axis=1)) +
                                (image - np.roll(image, -1, axis=0)) +
                                (image - np.roll(image, -1, axis=1)) < 0)

            image[change[:, 0], change[:, 1]] = pad_value
            tfill = 1 - (np.sum(image) / (image.shape[0] * image.shape[1]))

            if tfill >= fill:
                raise ValueError("Error: Unable to trim the image further.")

            fill = tfill
        image = 1 - image

    return image

In [64]:
def transform(image):
    return reshape(pad(crop(image)))

In [65]:
transformed_chars = {}
for name in chars.keys():
    transformed_chars[name] = transform(chars[name])
    # print(transformed_chars[name].shape)

print(len(transformed_chars))

fill_avg = []
for name in chars.keys():
    fill_avg.append(1- (np.sum(transformed_chars[name])/(transformed_chars[name].shape[0]*transformed_chars[name].shape[1])))
    
avg = np.mean(np.array(fill_avg))
print(avg)

if not os.path.exists('processesed_dataset'):
        os.makedirs('processesed_dataset')
        
for f in os.listdir('processesed_dataset'):
    if os.path.isfile('processesed_dataset/'+f): 
        os.remove('processesed_dataset/'+f)
    elif os.path.isdir('processesed_dataset/'+f):
        shutil.rmtree('processesed_dataset/'+f)
for name in chars.keys():
    if chars_labels[name] == '-':
        chars_labels.pop(name)
        transformed_chars.pop(name)
        continue
    
    transformed_chars[name] = trim(transformed_chars[name],avg)
    output_folder = 'processesed_dataset/'+chars_labels[name]
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cv2.imwrite(output_folder+'/'+name, transformed_chars[name]*255)


fill_avg = []
for name in transformed_chars.keys():
    fill_avg.append(1- (np.sum(transformed_chars[name])/(transformed_chars[name].shape[0]*transformed_chars[name].shape[1])))
    
avg = np.mean(np.array(fill_avg))
print(avg)


68751
0.23988900670535698
0.240473864597614


In [66]:
idx = np.random.choice(len(transformed_chars),len(transformed_chars)*8//10,replace=False)
names = transformed_chars.keys()

with open(trainfile, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([i for i in range(fnl_hyt*fnl_wdt + 1)])

    for i,name in enumerate(names):
        if i in idx:
            writer.writerow([ord(chars_labels[name])] + list(transformed_chars[name].flatten()))

In [67]:
with open(testfile, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([i for i in range(fnl_hyt*fnl_wdt + 1)])

    for i,name in enumerate(names):
        if i not in idx:
            writer.writerow([ord(chars_labels[name])] + list(transformed_chars[name].flatten()))

: 